In [16]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import dataset
import credentials
import sqlite3
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pickle
from Naive_bayes_model import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
f = open('nb.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [18]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [19]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [28]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener)
        
        stream.filter(track=['accident','fire','crash','ablaze','die','destroy'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']
        
        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [29]:
if __name__ == '__main__':
    
    hash_tag_list = ['accident','fire','crash','ablaze','die','destroy']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: RT @livia090657: Gefühle... 
können die Welt bedeuten, 
sie untergehen lassen 
und neu erwecken https://t.co/y49nF5Yuai

['rt', 'gefühle', '...', 'können', 'die', 'welt', 'bedeuten', 'sie', 'untergehen', 'lassen', 'und', 'neu', 'erwecken']

 Class: Relevant

Tweet: @HasnainKazim Könnten Sie sich im Gegenzug bitte mit der Dt. Gesch. ab 1905 auseinandersetzen, speziell mit den Asp… https://t.co/05vYsJYRme

['könnten', 'sie', 'sich', 'im', 'gegenzug', 'bitte', 'mit', 'der', 'dt', 'gesch', 'ab', '1905', 'auseinandersetzen', 'speziell', 'mit', 'den', 'asp…']

 Class: Can't Decide

Tweet: RT @M00Nblossom: wen ning is adorable and i would die for him okay? that's my boy https://t.co/ffJ5xcIMQO

['rt', 'wen', 'ning', 'adorable', 'would', 'die', 'okay', "'s", 'boy']

 Class: Can't Decide

Tweet: RT @BACKTAEYU: 🇺🇸 US GO
⚡ SUPER RUSH
PINS PINS PINS: WayV Moonwalk, Take Off, Dream Chewing Gum, Taeil Purple, Taeyong Long Flight, 127 Fir…

['rt', '🇺🇸', 'us', 'go', '⚡', 'super', 'rush', 'pins'

Tweet: @EAFussball @Sanchooo10 @BVB Momentum scripting anstossglitch und null ahnung von fussball. Eine schande an alle gr… https://t.co/1qJoRuI1MS

['momentum', 'scripting', 'anstossglitch', 'und', 'null', 'ahnung', 'von', 'fussball', 'eine', 'schande', 'alle', 'gr…']

 Class: Not Relevant

Tweet: Ebenfalls von zeitloser Schönheit: Die Situation in Belgien und den Niederlanden. Seit 1995 stabil: https://t.co/dbycjeHAPD

['ebenfalls', 'von', 'zeitloser', 'schönheit', 'die', 'situation', 'belgien', 'und', 'den', 'niederlanden', 'seit', '1995', 'stabil']

 Class: Can't Decide

Tweet: 🔵 Una dieta que incluya tomates disminuye el riesgo de contraer cáncer de hígado https://t.co/PdF9ZFyxlb

['🔵', 'una', 'dieta', 'que', 'incluya', 'tomates', 'disminuye', 'el', 'riesgo', 'de', 'contraer', 'cáncer', 'de', 'hígado']

 Class: Can't Decide

Tweet: 1910 erschien ein Buch unter dem Titel „Die Welt in 100 Jahren“. Die Beschreibung der mobilen Telefonie im Jahre 20… https://t.co/D4TNxbJWFe

['1910', 

['cinedimanche', 'comedie', 'diffuse', 'la', 'comédie', 'pattaya', 'de', 'franck', 'gastambide', 'à', '21h05']

 Class: Relevant

Tweet: RT @madhukishwar: While there are many instances of Islamists &amp;Christians committing genocides of non Muslims &amp;non Christians, only Communi…

['rt', 'many', 'instances', 'islamists', 'amp', 'christians', 'committing', 'genocides', 'non', 'muslims', 'amp', 'non', 'christians', 'communi…']

 Class: Can't Decide

Tweet: RT @BDSmovement: Israel uses @CaterpillarInc bulldozers to destroy Palestinian homes and agricultural land. 

Caterpillar also builds Israe…

['rt', 'israel', 'uses', 'bulldozers', 'destroy', 'palestinian', 'homes', 'agricultural', 'land', 'caterpillar', 'also', 'builds', 'israe…']

 Class: Can't Decide

Tweet: Just what i been looking for, a nigga added like 5 playlists

['looking', 'nigga', 'added', 'like', '5', 'playlists']

 Class: Can't Decide

Tweet: RT @PIRATEN_Saar: Moin Hamburg! In genau zwei Wochen ist Bürgerschaftswahl.

Tweet: @dpa_live @AZ_Augsburg Die FDP allein kann aber die Demokratie zerstören. Das hat sie ja meisterhaft hingekriegt.

['die', 'fdp', 'allein', 'kann', 'aber', 'die', 'demokratie', 'zerstören', 'das', 'hat', 'sie', 'ja', 'meisterhaft', 'hingekriegt']

 Class: Can't Decide

Tweet: Wenn Sie gerade lauten Jubel von Schulkindern hören, dann liegt das am #sturmsabine, der über Deutschland hinwegfeg… https://t.co/FTJ1eiG9Vr

['wenn', 'sie', 'gerade', 'lauten', 'jubel', 'von', 'schulkindern', 'hören', 'dann', 'liegt', 'das', 'sturmsabine', 'der', 'über', 'deutschland', 'hinwegfeg…']

 Class: Not Relevant



InterfaceError: Error binding parameter 19 - probably unsupported type.